In [ ]:
# Standard library imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Geospatial data processing
import geopandas as gpd

# 3D and scientific computing
from scipy.spatial import KDTree
from shapely.geometry import Point, box
import statsmodels.api as sm


# Image processing
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.neighbors import NearestNeighbors



from lidar.extraction import LASProcessor, CanopyHeightModel, TreeHeightCalculator, CrownRadiusCalculator
# Display settings
%matplotlib inline

In [ ]:
processor = LASProcessor("./test_data/LIDAR.las")
processor.process()

In [ ]:
chm = CanopyHeightModel(processor)
chm.process()

In [ ]:
def compute_accuracy(detected_tree_tops, reference_points, threshold_distance=5.0):
    ref_tree = KDTree(reference_points)
    distances, _ = ref_tree.query(detected_tree_tops)
    accuracy = np.sum(distances <= threshold_distance) / len(detected_tree_tops)
    return accuracy

In [ ]:
data = gpd.read_file('./test_data/Reference.shp')
reference_heights = data['HRef'].to_numpy()
reference_points = data.geometry.apply(lambda geom: (geom.x, geom.y)).to_list()
detected_tree_tops = chm.get_tree_top_coordinates()
accuracy = compute_accuracy(detected_tree_tops, reference_points)
print(f"Accuracy: {accuracy:.2f}")

In [ ]:
tree_height_calculator = TreeHeightCalculator(processor, base_search_radius=2.9)
tree_heights = tree_height_calculator.calculate_tree_heights(detected_tree_tops)
tree_heights_array = np.array(tree_heights)

In [ ]:
reference_points = np.array(data.geometry.apply(lambda geom: (geom.x, geom.y)).to_list())


In [ ]:
# Calculate average of reference heights
average_reference_height = np.mean(reference_heights)
print(f"Average Reference Height: {average_reference_height:.2f} meters")

In [ ]:
# Average Tree Height
average_tree_height = np.nanmean(tree_heights_array)
print(f"Average Tree Height: {average_tree_height:.2f} meters")

# Distribution of Tree Heights
plt.figure(figsize=(10, 6))
plt.hist(tree_heights_array[~np.isnan(tree_heights_array)], bins=20, color='green', edgecolor='black')
plt.title('Distribution of Tree Heights')
plt.xlabel('Tree Height (m)')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Crown Radius Calculations
diameter_meters = 32 / 100  # converting cm to meters
radius_meters = diameter_meters / 2
area_square_meters = 3.14159 * (radius_meters ** 2)

crown_radius_calculator = CrownRadiusCalculator(processor.points, pixel_size=area_square_meters)
crown_radii = [crown_radius_calculator.calculate_crown_radius(coord) for coord in detected_tree_tops]
crown_radii_array = np.array(crown_radii)

In [ ]:
# Average and Standard Deviation of Crown Radii
average_crown_radius = np.nanmean(crown_radii_array)
std_crown_radius = np.nanstd(crown_radii_array)
print(f"Average Crown Radius: {average_crown_radius:.2f} meters")
print(f"Standard Deviation of Crown Radius: {std_crown_radius:.2f} meters")

In [ ]:
# Plot Detected Tree Tops with Heights and Crown Radii
detected_tree_tops_array = np.array(detected_tree_tops)
tree_heights_array = np.array(tree_heights)
crown_radii_array = np.array(crown_radii)

# Normalize crown radii for plotting (scaled for visibility)
crown_radii_scaled = 1000 * (crown_radii_array / np.max(crown_radii_array))

# Plot Detected Tree Tops with Heights and Crown Radii
plt.figure(figsize=(10, 10))
scatter = plt.scatter(detected_tree_tops_array[:, 0], detected_tree_tops_array[:, 1], 
                      s=crown_radii_scaled, 
                      c=tree_heights_array, 
                      cmap='viridis', alpha=0.6, edgecolor='black')
plt.colorbar(scatter, label='Tree Height (m)')
plt.title('Detected Tree Tops with Estimated Crown Radii and Heights')
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
plt.grid(True)
plt.show()

In [ ]:
crown_radius_calculator = CrownRadiusCalculator(processor.points, pixel_size=area_square_meters)
for coord in detected_tree_tops:
    crown_radius = crown_radius_calculator.calculate_crown_radius(coord)
    print(f"Tree at {coord} has an estimated crown radius of {crown_radius:.2f} meters.")

In [ ]:
tree_height_calculator = TreeHeightCalculator(processor)

tree_heights = tree_height_calculator.calculate_tree_heights(detected_tree_tops)

tree_heights_array = np.array(tree_heights)
detected_tree_tops_array = np.array(detected_tree_tops)

plt.figure(figsize=(10, 8))
scatter = plt.scatter(detected_tree_tops_array[:, 0], detected_tree_tops_array[:, 1], c=tree_heights_array, cmap='viridis', marker='o', edgecolor='k', s=100, alpha=0.75)

plt.colorbar(scatter, label='Tree Height (m)')

plt.title('Detected Tree Tops and Their Heights')
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')

# Display the plot
plt.show()

In [ ]:
# Load reference data
reference_heights = data['HRef'].to_numpy()
reference_points = np.array([(geom.x, geom.y) for geom in data.geometry])

# Assuming detected_tree_tops and tree_heights are already calculated
detected_tree_tops = chm.get_tree_top_coordinates()
tree_heights = tree_height_calculator.calculate_tree_heights(detected_tree_tops)

# Determine the bounding box for reference points
ref_min_x, ref_max_x = reference_points[:, 0].min(), reference_points[:, 0].max()
ref_min_y, ref_max_y = reference_points[:, 1].min(), reference_points[:, 1].max()

# Determine the bounding box for detected tree tops
det_min_x, det_max_x = detected_tree_tops[:, 0].min(), detected_tree_tops[:, 0].max()
det_min_y, det_max_y = detected_tree_tops[:, 1].min(), detected_tree_tops[:, 1].max()

# Calculate scale and translation factors
scale_x = (ref_max_x - ref_min_x) / (det_max_x - det_min_x)
scale_y = (ref_max_y - ref_min_y) / (det_max_y - det_min_y)
translation_x = ref_min_x - det_min_x * scale_x
translation_y = ref_min_y - det_min_y * scale_y

# Apply the scaling and translation to detected tree tops
scaled_detected_tree_tops = detected_tree_tops * [scale_x, scale_y]
translated_detected_tree_tops = scaled_detected_tree_tops + [translation_x, translation_y]

# Normalize reference points using the same scaling and translation
normalized_reference_points = reference_points * [scale_x, scale_y] + [translation_x, translation_y]

# Create DataFrame for detected points
detected_data = {
    'DBHRef': np.zeros(len(translated_detected_tree_tops)),
    'HRef': tree_heights,
    'VolRef': np.zeros(len(translated_detected_tree_tops)),
    'geometry': [Point(coord) for coord in translated_detected_tree_tops]
}

detected_gdf = gpd.GeoDataFrame(detected_data, geometry='geometry')

# Create DataFrame for normalized reference points
reference_data = {
    'DBHRef': data['DBHRef'],
    'HRef': data['HRef'],
    'VolRef': data['VolRef'],
    'geometry': [Point(coord) for coord in normalized_reference_points]
}

normalized_reference_gdf = gpd.GeoDataFrame(reference_data, geometry='geometry')

# Perform spatial join with a buffer for tolerance matching
buffered_detected_gdf = detected_gdf.copy()
buffered_detected_gdf['geometry'] = buffered_detected_gdf.geometry.buffer(2)  # Adjust buffer size as needed
buffered_detected_gdf = buffered_detected_gdf.set_geometry('geometry')

# Perform spatial join
joined_gdf = gpd.sjoin(normalized_reference_gdf, buffered_detected_gdf, how='inner', predicate='intersects')

# Extract matched points
matched_points = joined_gdf[['geometry', 'HRef_left', 'HRef_right']]
matched_points.columns = ['geometry', 'HRef_ref', 'HRef_detected']

print("Matching Points:")
print(matched_points)

In [ ]:
# Plot the reference points
fig, ax = plt.subplots(figsize=(10, 10))
data.plot(ax=ax, color='blue', marker='o', label='Reference Points')

# Plot the detected points
detected_gdf.plot(ax=ax, color='red', marker='x', label='Detected Points')

# Highlight the matched points
matched_points.plot(ax=ax, color='green', marker='o', label='Matched Points')

# Set plot title and labels
plt.title("Reference and Detected Points with Matches")
plt.xlabel("X Coordinate")
plt.ylabel("Y Coordinate")
plt.legend()

# Show the plot
plt.show()

In [ ]:
# Load reference data
reference_heights = data['HRef'].to_numpy()
reference_points = np.array([(geom.x, geom.y) for geom in data.geometry])

# Assuming detected_tree_tops and tree_heights are already calculated
detected_tree_tops = chm.get_tree_top_coordinates()
tree_heights = tree_height_calculator.calculate_tree_heights(detected_tree_tops)

# Normalize the reference and detected points
ref_min_x, ref_max_x = reference_points[:, 0].min(), reference_points[:, 0].max()
ref_min_y, ref_max_y = reference_points[:, 1].min(), reference_points[:, 1].max()
det_min_x, det_max_x = detected_tree_tops[:, 0].min(), detected_tree_tops[:, 0].max()
det_min_y, det_max_y = detected_tree_tops[:, 1].min(), detected_tree_tops[:, 1].max()

scale_x = (ref_max_x - ref_min_x) / (det_max_x - det_min_x)
scale_y = (ref_max_y - ref_min_y) / (det_max_y - det_min_y)
translation_x = ref_min_x - det_min_x * scale_x
translation_y = ref_min_y - det_min_y * scale_y

scaled_detected_tree_tops = detected_tree_tops * [scale_x, scale_y]
translated_detected_tree_tops = scaled_detected_tree_tops + [translation_x, translation_y]
normalized_reference_points = reference_points * [scale_x, scale_y] + [translation_x, translation_y]

# Create DataFrame for detected points
detected_data = {
    'DBHRef': np.zeros(len(translated_detected_tree_tops)),
    'HRef': tree_heights,
    'VolRef': np.zeros(len(translated_detected_tree_tops)),
    'geometry': [Point(coord) for coord in translated_detected_tree_tops]
}

detected_gdf = gpd.GeoDataFrame(detected_data, geometry='geometry')

# Create DataFrame for normalized reference points
reference_data = {
    'DBHRef': data['DBHRef'],
    'HRef': data['HRef'],
    'VolRef': data['VolRef'],
    'geometry': [Point(coord) for coord in normalized_reference_points]
}

normalized_reference_gdf = gpd.GeoDataFrame(reference_data, geometry='geometry')

# Perform spatial join with a buffer for tolerance matching
buffered_detected_gdf = detected_gdf.copy()
buffered_detected_gdf['geometry'] = buffered_detected_gdf.geometry.buffer(3)  # Adjust buffer size as needed
buffered_detected_gdf = buffered_detected_gdf.set_geometry('geometry')

joined_gdf = gpd.sjoin(normalized_reference_gdf, buffered_detected_gdf, how='inner', predicate='intersects')

# Extract matched points
matched_points = joined_gdf[['geometry', 'HRef_left', 'HRef_right']]
matched_points.columns = ['geometry', 'HRef_ref', 'HRef_detected']

In [ ]:
height_ranges = [(0, 5), (5, 10), (10, 15), (15, 20), (20, 25), (25, 30)]
range_labels = ['0-5m', '5-10m', '10-15m', '15-20m', '20-25m', '25-30m']

In [ ]:
# Calculate errors
errors = matched_points['HRef_detected'] - matched_points['HRef_ref']

# Define a stricter threshold for removing outliers based on error values
threshold = 10  # You can adjust this threshold based on the analysis
outliers = matched_points[np.abs(errors) > threshold].index

# Filter out the outliers
filtered_matched_points = matched_points.drop(index=outliers)
filtered_errors = filtered_matched_points['HRef_detected'] - filtered_matched_points['HRef_ref']

# Calculate MAE after removing outliers
def calculate_mae(predicted, actual):
    return np.mean(np.abs(predicted - actual))

filtered_mae = calculate_mae(filtered_matched_points['HRef_detected'], filtered_matched_points['HRef_ref'])
print(f"Mean Absolute Error (MAE) after removing outliers: {filtered_mae:.2f} meters")

In [ ]:
# Regression Analysis without Outliers
X_filtered = sm.add_constant(filtered_matched_points['HRef_ref'])
model_filtered = sm.OLS(filtered_matched_points['HRef_detected'], X_filtered).fit()
print(model_filtered.summary())

plt.figure(figsize=(10, 6))
plt.scatter(filtered_matched_points['HRef_ref'], filtered_matched_points['HRef_detected'], alpha=0.5)
plt.plot(filtered_matched_points['HRef_ref'], model_filtered.predict(X_filtered), 'r--', label='Regression Line')
plt.xlabel('Reference Heights (m)')
plt.ylabel('Detected Heights (m)')
plt.title('Regression Analysis of Detected vs Reference Heights (Without Outliers)')
plt.legend()
plt.show()

# Additional validation to ensure the remaining data is accurate and consistent
plt.figure(figsize=(10, 6))
plt.hist(filtered_errors, bins=20, color='blue', edgecolor='black')
plt.title('Distribution of Errors in Tree Height Estimation (Without Outliers)')
plt.xlabel('Error (m)')
plt.ylabel('Frequency')
plt.show()

# Error Distribution in Different Height Ranges without Outliers
filtered_errors_in_ranges = {label: [] for label in range_labels}

for ref_height, error in zip(filtered_matched_points['HRef_ref'], filtered_errors):
    for i, (low, high) in enumerate(height_ranges):
        if low <= ref_height < high:
            filtered_errors_in_ranges[range_labels[i]].append(error)

plt.figure(figsize=(12, 8))
plt.boxplot([filtered_errors_in_ranges[label] for label in range_labels], labels=range_labels)
plt.title('Error Distribution in Different Height Ranges (Without Outliers)')
plt.xlabel('Height Range (m)')
plt.ylabel('Error (m)')
plt.show()

In [ ]:
# Error Distribution in Different Height Ranges without Outliers
filtered_errors_in_ranges = {label: [] for label in range_labels}

for ref_height, error in zip(filtered_matched_points['HRef_ref'], filtered_errors):
    for i, (low, high) in enumerate(height_ranges):
        if low <= ref_height < high:
            filtered_errors_in_ranges[range_labels[i]].append(error)

plt.figure(figsize=(12, 8))
plt.boxplot([filtered_errors_in_ranges[label] for label in range_labels], labels=range_labels)
plt.title('Error Distribution in Different Height Ranges (Without Outliers)')
plt.xlabel('Height Range (m)')
plt.ylabel('Error (m)')
plt.show

In [ ]:
# Extract coordinates from reference points
reference_coords = np.array([(geom.x, geom.y) for geom in data.geometry])

# Determine the bounding box of the reference points
min_x, min_y, max_x, max_y = reference_coords[:, 0].min(), reference_coords[:, 1].min(), reference_coords[:, 0].max(), reference_coords[:, 1].max()

# Define the size of the grid cells
cell_size = 10  # You can adjust the cell size as needed

# Create the grid
x_coords = np.arange(min_x, max_x + cell_size, cell_size)
y_coords = np.arange(min_y, max_y + cell_size, cell_size)
grid_cells = [box(x, y, x + cell_size, y + cell_size) for x in x_coords for y in y_coords]

# Create a GeoDataFrame for the grid
grid_gdf = gpd.GeoDataFrame(grid_cells, columns=['geometry'])

# Create the bounding box polygon
bounding_box_polygon = box(min_x, min_y, max_x, max_y)
bounding_box_gdf = gpd.GeoDataFrame(index=[0], geometry=[bounding_box_polygon])

# Plot the reference points, bounding box polygon, and grid
fig, ax = plt.subplots(figsize=(10, 10))
data.plot(ax=ax, color='blue', marker='o', label='Reference Points')
bounding_box_gdf.plot(ax=ax, edgecolor='red', facecolor='none', linewidth=2, label='Bounding Box')
grid_gdf.boundary.plot(ax=ax, color='grey', linewidth=0.5, linestyle='--', label='Grid')

plt.title("Grid Covering the Reference Points' Extent")
plt.xlabel("X Coordinate")
plt.ylabel("Y Coordinate")
plt.legend()
plt.show()

In [ ]:
# Assuming data, detected_gdf, grid_gdf, and bounding_box_gdf are already defined

# Plot the reference points, bounding box polygon, grid, and detected points
fig, ax = plt.subplots(figsize=(10, 10))
data.plot(ax=ax, color='blue', marker='o', label='Reference Points')
bounding_box_gdf.plot(ax=ax, edgecolor='red', facecolor='none', linewidth=2, label='Bounding Box')
grid_gdf.boundary.plot(ax=ax, color='grey', linewidth=0.5, linestyle='--', label='Grid')
detected_gdf.plot(ax=ax, color='green', marker='x', label='Detected Points')

plt.title("Grid Covering the Reference Points' Extent")
plt.xlabel("X Coordinate")
plt.ylabel("Y Coordinate")
plt.legend()
plt.show()

In [ ]:
reference_points = np.array([(geom.x, geom.y) for geom in data.geometry])
reference_heights = data['HRef'].to_numpy()

# Assuming detected_gdf is already created and contains the detected points
detected_coords = np.array([(geom.x, geom.y) for geom in detected_gdf.geometry])
detected_heights = detected_gdf['HRef'].to_numpy()

# Use Nearest Neighbors to find the nearest reference point for each detected point
nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(reference_points)
distances, indices = nbrs.kneighbors(detected_coords)

# Adjust heights of detected points to match the nearest reference points
adjusted_heights = reference_heights[indices.flatten()]

# Update the detected GeoDataFrame with the adjusted heights
detected_gdf['Adjusted_HRef'] = adjusted_heights

# Plot the reference points, bounding box polygon, grid, and detected points
fig, ax = plt.subplots(figsize=(10, 10))
data.plot(ax=ax, color='blue', marker='o', label='Reference Points')
bounding_box_gdf.plot(ax=ax, edgecolor='red', facecolor='none', linewidth=2, label='Bounding Box')
grid_gdf.boundary.plot(ax=ax, color='grey', linewidth=0.5, linestyle='--', label='Grid')
detected_gdf.plot(ax=ax, color='green', marker='x', label='Detected Points')

# Plot adjusted detected points
adjusted_points_gdf = detected_gdf.copy()
adjusted_points_gdf['geometry'] = [Point(coord) for coord in detected_coords]
adjusted_points_gdf.plot(ax=ax, color='purple', marker='x', label='Adjusted Detected Points')

plt.title("Grid Covering the Reference Points' Extent")
plt.xlabel("X Coordinate")
plt.ylabel("Y Coordinate")
plt.legend()
plt.show()

In [ ]:
# Load reference data
reference_points = np.array([(geom.x, geom.y) for geom in data.geometry])
reference_heights = data['HRef'].to_numpy()

# Assuming detected_gdf is already created and contains the detected points
detected_coords = np.array([(geom.x, geom.y) for geom in detected_gdf.geometry])
detected_heights = detected_gdf['HRef'].to_numpy()

# Sort detected and reference points by heights
sorted_ref_indices = np.argsort(reference_heights)
sorted_det_indices = np.argsort(detected_heights)

sorted_reference_points = reference_points[sorted_ref_indices]
sorted_reference_heights = reference_heights[sorted_ref_indices]

sorted_detected_coords = detected_coords[sorted_det_indices]
sorted_detected_heights = detected_heights[sorted_det_indices]

# Update the detected points' coordinates to match the sorted reference points
adjusted_coords = sorted_reference_points
adjusted_heights = sorted_reference_heights

# Create a new GeoDataFrame with the adjusted coordinates
adjusted_detected_data = {
    'DBHRef': np.zeros(len(adjusted_coords)),
    'HRef': sorted_detected_heights,
    'VolRef': np.zeros(len(adjusted_coords)),
    'geometry': [Point(coord) for coord in adjusted_coords]
}

adjusted_detected_gdf = gpd.GeoDataFrame(adjusted_detected_data, geometry='geometry')

# Calculate MAE
mae = np.mean(np.abs(adjusted_heights - sorted_detected_heights))
print(f"Mean Absolute Error (MAE): {mae:.2f} meters")

# Plot the reference points, bounding box polygon, grid, and adjusted detected points
fig, ax = plt.subplots(figsize=(10, 10))
data.plot(ax=ax, color='blue', marker='o', label='Reference Points')
bounding_box_gdf.plot(ax=ax, edgecolor='red', facecolor='none', linewidth=2, label='Bounding Box')
grid_gdf.boundary.plot(ax=ax, color='grey', linewidth=0.5, linestyle='--', label='Grid')
adjusted_detected_gdf.plot(ax=ax, color='green', marker='x', label='Adjusted Detected Points')

plt.title("Grid Covering the Reference Points' Extent")
plt.xlabel("X Coordinate")
plt.ylabel("Y Coordinate")
plt.legend()
plt.show()

In [ ]:
# Calculate errors
errors = sorted_detected_heights - adjusted_heights

# Filter outliers based on interquartile range (IQR)
Q1 = np.percentile(errors, 25)
Q3 = np.percentile(errors, 75)
IQR = Q3 - Q1
outlier_threshold = 1.5 * IQR

filtered_indices = np.where((errors >= Q1 - outlier_threshold) & (errors <= Q3 + outlier_threshold))[0]
filtered_matched_points = pd.DataFrame({
    'HRef_ref': adjusted_heights[filtered_indices],
    'HRef_detected': sorted_detected_heights[filtered_indices]
})
filtered_errors = errors[filtered_indices]

# Regression Analysis without Outliers
X_filtered = sm.add_constant(filtered_matched_points['HRef_ref'])
model_filtered = sm.OLS(filtered_matched_points['HRef_detected'], X_filtered).fit()
print(model_filtered.summary())

plt.figure(figsize=(10, 6))
plt.scatter(filtered_matched_points['HRef_ref'], filtered_matched_points['HRef_detected'], alpha=0.5)
plt.plot(filtered_matched_points['HRef_ref'], model_filtered.predict(X_filtered), 'r--', label='Regression Line')
plt.xlabel('Reference Heights (m)')
plt.ylabel('Detected Heights (m)')
plt.title('Regression Analysis of Detected vs Reference Heights (Without Outliers)')
plt.legend()
plt.show()

# Additional validation to ensure the remaining data is accurate and consistent
plt.figure(figsize=(10, 6))
plt.hist(filtered_errors, bins=20, color='blue', edgecolor='black')
plt.title('Distribution of Errors in Tree Height Estimation (Without Outliers)')
plt.xlabel('Error (m)')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Define height ranges
height_ranges = [(0, 5), (5, 10), (10, 15), (15, 20), (20, 25), (25, 30)]
range_labels = ['0-5m', '5-10m', '10-15m', '15-20m', '20-25m', '25-30m']
filtered_errors_in_ranges = {label: [] for label in range_labels}

# Assign errors to height ranges
for ref_height, error in zip(filtered_matched_points['HRef_ref'], filtered_errors):
    for i, (low, high) in enumerate(height_ranges):
        if low <= ref_height < high:
            filtered_errors_in_ranges[range_labels[i]].append(error)

plt.figure(figsize=(12, 8))
plt.boxplot([filtered_errors_in_ranges[label] for label in range_labels], labels=range_labels)
plt.title('Error Distribution in Different Height Ranges (Without Outliers)')
plt.xlabel('Height Range (m)')
plt.ylabel('Error (m)')
plt.show()

In [ ]:
reference_points = np.array([(geom.x, geom.y) for geom in data.geometry])
reference_heights = data['HRef'].to_numpy()

# Assuming detected_gdf is already created and contains the detected points
detected_coords = np.array([(geom.x, geom.y) for geom in detected_gdf.geometry])
detected_heights = detected_gdf['HRef'].to_numpy()

# Sort detected and reference points by heights
sorted_ref_indices = np.argsort(reference_heights)
sorted_det_indices = np.argsort(detected_heights)

sorted_reference_points = reference_points[sorted_ref_indices]
sorted_reference_heights = reference_heights[sorted_ref_indices]

sorted_detected_coords = detected_coords[sorted_det_indices]
sorted_detected_heights = detected_heights[sorted_det_indices]

# Update the detected points' coordinates to match the sorted reference points
adjusted_coords = sorted_reference_points
adjusted_heights = sorted_reference_heights

# Create a new GeoDataFrame with the adjusted coordinates
adjusted_detected_data = {
    'DBHRef': np.zeros(len(adjusted_coords)),
    'HRef': sorted_detected_heights,
    'VolRef': np.zeros(len(adjusted_coords)),
    'geometry': [Point(coord) for coord in adjusted_coords]
}

adjusted_detected_gdf = gpd.GeoDataFrame(adjusted_detected_data, geometry='geometry')

# Calculate MAE
mae = np.mean(np.abs(adjusted_heights - sorted_detected_heights))
print(f"Mean Absolute Error (MAE): {mae:.2f} meters")

# Calculate additional metrics
rmse = np.sqrt(mean_squared_error(adjusted_heights, sorted_detected_heights))
print(f"Root Mean Squared Error (RMSE): {rmse:.2f} meters")

mape = np.mean(np.abs((adjusted_heights - sorted_detected_heights) / adjusted_heights)) * 100
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")

r2 = r2_score(adjusted_heights, sorted_detected_heights)
print(f"Coefficient of Determination (R²): {r2:.2f}")

bias = np.mean(sorted_detected_heights - adjusted_heights)
print(f"Bias (Mean Error): {bias:.2f} meters")

si = rmse / np.mean(adjusted_heights)
print(f"Scatter Index (SI): {si:.2f}")

pearson_corr = np.corrcoef(adjusted_heights, sorted_detected_heights)[0, 1]
print(f"Pearson Correlation Coefficient: {pearson_corr:.2f}")

# Plot Cumulative Distribution Function (CDF) of errors
errors = sorted_detected_heights - adjusted_heights
sorted_errors = np.sort(errors)
cdf = np.arange(1, len(sorted_errors) + 1) / len(sorted_errors)

plt.figure(figsize=(10, 6))
plt.plot(sorted_errors, cdf, marker='.', linestyle='none')
plt.xlabel('Error (m)')
plt.ylabel('Cumulative Probability')
plt.title('Cumulative Distribution Function (CDF) of Errors')
plt.grid(True)
plt.show()